# Uncertainty-Aware Road Obstacle Identification

## 1. Imports
Import necessary libraries like PyTorch, torchvision, etc.

In [1]:
import sys
print(sys.executable)

c:\Users\marce\3D Objects\Sapienza\Computer Vision\maybe-obstacle\venv\Scripts\python.exe


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

## 2. Globals
Define global variables such as paths, batch size, learning rate, etc.

In [2]:
# Global variables
DATASET_PATH = "./datasets/"
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EPOCHS = 50

# Select the best available device (CPU, CUDA, or MPS (Metal Performance Shaders for macOS))
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using CUDA device: {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Metal Performance Shaders) device")
else:
    device = torch.device("cpu")
    print("Using CPU")
DEVICE = device

Using CPU


## 3. Utils
Helper functions for visualization, preprocessing, and more.

In [ ]:

def visualize_sample(image, mask):
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Image")
    plt.imshow(image)
    plt.subplot(1, 2, 2)
    plt.title("Mask")
    plt.imshow(mask)
    plt.show()


### 3.1 Utils for Conformal Risk Control

Miscoverage Loss

In [ ]:
"""
    Compute l(Z, Y) = 1 - (# correctly covered pixels) / (# total pixels).

    Parameters
    ----------
    set_mask : binary tensor, shape [C, H, W]
        For every pixel (i,j) and class k: 1 if class k is included in the set S_ij, 0 otherwise.
    labels   : int tensor, shape [H, W]
        Ground-truth class index per pixel (values in 0 … C-1).

    Returns
    -------
    loss : scalar tensor
        Miscoverage rate in the current image (float in [0, 1]).
"""
def miscoverage_loss(set_mask: torch.Tensor,  # [C, H, W] 0/1   – predicted set
                     labels:    torch.Tensor  # [H, W]       int – ground-truth class indices
                     ) -> torch.Tensor:
   
    C, H, W = set_mask.shape                       # number of classes & image size

    # Build a one-hot tensor for the ground-truth mask Y
    # gt_mask[k,i,j] = 1 if ground-truth class at (i,j) == k
    gt_mask = torch.zeros_like(set_mask)
    gt_mask.scatter_(0, labels.unsqueeze(0), 1)     # in-place one-hot

    # Pixel is covered if gt class ∈ predicted set
    covered = (set_mask * gt_mask).sum(dim=0)       # → [H, W] ∈ {0,1}

    # Coverage ratio = mean over all pixels
    coverage = covered.float().mean()               # scalar

    # Miscoverage loss ℓ = 1 − coverage
    return 1.0 - coverage


Least Ambiguous Set-Valued Classifiers

In [4]:
def T_lambda(probs, lam):
    # probs: [C, H, W], lam: float ∈ [0,1]
    return (probs >= (1 - lam)).float()  # binariza por canal

Dichotomic search over the parameter λ

In [ ]:

"""
We can compute the optimal like this because the Loss is monotonic in lambda
-  Lambda is the threshold for binarization in T_lambda
-  Alpha is the tolerated risk level
-  Eps is the precision for the binary search
"""

def binary_search_lambda(probs_list,labels_list,alpha=0.1,eps=1e-3):
    left , right = 0.0, 1.0
    while right - left > eps:
        mid = (left + right) / 2.0
        avr_risk = 0.0 
        for probs,labels in zip(probs_list,labels_list):
            mask = T_lambda(probs, mid)
            risk += miscoverage_loss(mask,labels)        
        avr_risk += risk / len(probs_list)
        if avr_risk < alpha:
            right = mid
        else:
            left = mid
    return mid

## 4. Data
Load and preprocess datasets like Cityscapes, LostAndFound, and Fishyscapes.

In [ ]:
# Code for data loading and preprocessing

## 5. Network
Define the neural network architecture for semantic segmentation.

In [ ]:
# Define the model architecture here

# Lets replicate the Unknown Objectness Scores setup

# Load a pretrained DeepLabv3 model with a ResNet-50 backbone
model = models.segmentation.deeplabv3_resnet50(pretrained=True)
model = model.to(DEVICE)
model.eval()  # Set the model to evaluation mode


### Conformal Risk Control

#### Calibration

1. Pass the calibration set through the model to get logits
2. Convert logits to probabilities using softmax.
3. Run binary_search_lambda over the calibration probabilities to find the optimal Lambda

#### Implementation
1. Pass the test set through the model to get logits and compute softmax
2. Use optimal Lambda to get mask
3. Visualize the cardinality of the set. (Pixel wise anomaly detection measurement)


## 6. Train
Implement the training loop, including loss functions and optimizers.

In [ ]:
# Training loop and loss computation

## 7. Test
Test the trained model and evaluate performance metrics.

In [ ]:
# Evaluation metrics and performance analysis